In [1]:
# Libraries used
import time
import torch
import torch.utils.data as data_utils
import torch.autograd as ag
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from resnet import ResNet
from data_handler import Data_Handler

# Loads the directory that contains patched images for batches
data_handler = Data_Handler('mass_buildings', 'train')

In [2]:
net = ResNet(layers = 34)
dataset = data_handler.create_dataset() # Temporary, only used to get sample count

if torch.cuda.is_available():
    net = net.cuda()
    
weight = torch.ones(2)
weight_decay = 2e-4 
weight[1] = 0

N = len(dataset) # Training set size
B = 50 # Minibacth size
NB = (N+B-1)/B # Number of minibatches
T = 5 # Number of epochs
gamma = .0002 # learning rate
rho = .9 # momentum

if torch.cuda.is_available():
    criterion = nn.CrossEntropyLoss(weight.cuda())
else:
    criterion = nn.CrossEntropyLoss(weight)
optimizer = torch.optim.SGD(net.parameters(),
                            lr = gamma,
                            momentum = rho)

In [3]:
for epoch in range(T):
    # Reloads data directory filenames
    dataset = data_handler.create_dataset()
    
    for k in range(NB):
        
        # Pops files in dataset and imports them as torch tensor 
        batch = data_handler.create_batch(dataset, B) 
        running_loss = 0.0
        inputs = ag.Variable(batch[0].type(torch.FloatTensor), requires_grad = True)
        labels = ag.Variable(batch[1].type(torch.LongTensor), requires_grad = False)
        
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
    
        # Initialize the gradients to zero
        optimizer.zero_grad()

        # Forward propagation
        outputs = net(inputs)

        # Error evaluation
        loss = criterion(outputs, labels)

        # Back propagation
        loss.backward()

        # Parameter update
        optimizer.step()

        # Print averaged loss per minibatch every 100 mini - batches
        running_loss += loss[0]
        if k % 10 == 9:
            print('[Epoch: %d, Batch: %3d /%3d] loss : %.3f' % (epoch + 1, k + 1, NB,running_loss/B))
            running_loss = 0.0
    
print('Finished Training')

[Epoch: 1, Batch:  10 / 99] loss : 0.014
[Epoch: 1, Batch:  20 / 99] loss : 0.013
[Epoch: 1, Batch:  30 / 99] loss : 0.013
[Epoch: 1, Batch:  40 / 99] loss : 0.013
[Epoch: 1, Batch:  50 / 99] loss : 0.012
[Epoch: 1, Batch:  60 / 99] loss : 0.011
[Epoch: 1, Batch:  70 / 99] loss : 0.011
[Epoch: 1, Batch:  80 / 99] loss : 0.011
[Epoch: 1, Batch:  90 / 99] loss : 0.010
[Epoch: 2, Batch:  10 / 99] loss : 0.009
[Epoch: 2, Batch:  20 / 99] loss : 0.009
[Epoch: 2, Batch:  30 / 99] loss : 0.009
[Epoch: 2, Batch:  40 / 99] loss : 0.009
[Epoch: 2, Batch:  50 / 99] loss : 0.008
[Epoch: 2, Batch:  60 / 99] loss : 0.008
[Epoch: 2, Batch:  70 / 99] loss : 0.008
[Epoch: 2, Batch:  80 / 99] loss : 0.008
[Epoch: 2, Batch:  90 / 99] loss : 0.007
[Epoch: 3, Batch:  10 / 99] loss : 0.007
[Epoch: 3, Batch:  20 / 99] loss : 0.007
[Epoch: 3, Batch:  30 / 99] loss : 0.007
[Epoch: 3, Batch:  40 / 99] loss : 0.006
[Epoch: 3, Batch:  50 / 99] loss : 0.006
[Epoch: 3, Batch:  60 / 99] loss : 0.006
[Epoch: 3, Batch

In [4]:
torch.save(net, 'buildings_model.pkl')